In [4]:
import sys
sys.path.append(r'D:\学习\微热点大赛\DataMiningForCompetation')
from db import MyDB
#加载数据库
db = MyDB(host='182.61.54.181', user='root', password='yW88224646!', db='WRD')

In [1]:
#加载使用库
import pandas
import os
import numpy
import jieba.posseg as pseg
import re
from tqdm import tqdm
import json
import threading

In [2]:
def saveTempFile(path,Object_):
    with open(path,'w') as f:
        json.dump(Object_,f)


In [2]:
# [是否为默认文本,内容文本,内容表情,地域,城市,性别,粉丝数,微博数,转,评,赞,话题,微博情绪,精准地域,中图地址,
# MD5-作者ID,MD5-mid,MD5-根微博mid,MD5-根微博用户UID,MD5-父微博ID,MD5-父微博用户ID]
targetRoot = r'E:\Download\数据大赛\选题1\初步分词\\'
sourceRoot = r'E:\Download\数据大赛\选题1\微博'
needPartOfSpeech = ['a','ad','an']
tl1 = []
emotion = re.compile(r'(\[.*?\])')
tag = re.compile(r'(#.*?#)')
tags = []
emotions = []
files = None
for root,dirs,fs in os.walk(sourceRoot):
    files = fs
    break

In [ ]:
for file in files:
    fileAbsolutePath = sourceRoot+'\\'+file
    dataFile = pandas.read_csv(fileAbsolutePath,encoding='gb18030')
    tempList = []
    for item in tqdm(dataFile.values):
        temp = [int(item[0] == '转发微博 '),json.dumps(emotion.findall(item[0])), json.dumps(tag.findall(item[0])), item[0]]
        for e in temp[1]:
            temp[3] = temp[3].replace(e,'')
        for t in temp[2]:
            temp[3] = temp[3].replace(t,'')
        if temp[3] == '转发微博 ':
            temp[3] = ''
        tempWords = []
        words = pseg.cut(temp[3],use_paddle=True)
        for word, flag in words:
            if flag in needPartOfSpeech:
                tempWords.append(word)
        temp[3] = json.dumps(tempWords)
        temp.append(int(item[1] == '敏感'))
        temp.append(int(item[2] == '原创'))
        for i in range(3,25):
            if i == 9 or i == 4:
                continue
            temp.append(item[i])
        tempList.append(temp)
    tempList = pandas.DataFrame(tempList)
    tempList.columns = ['是否为默认文本','内容表情','内容标签','过滤后内容分词json串','是否敏感','是否原创','发布日期','认证类型','地域','城市','性别','粉丝数','微博数','转','评','赞','话题','微博情绪','精准地域','中图地址',
      'MD5-作者ID','MD5-mid','MD5-根微博mid','MD5-根微博用户UID','MD5-父微博ID','MD5-父微博用户ID']
    tempList.to_csv(targetRoot+file,encoding='gb18030')

In [3]:
#读取一个csv文件进行分词结果分析
test = pandas.read_csv(r'E:\Download\数据大赛\选题1\初步分词\《乘风破浪的姐姐》成团之夜-2.csv',encoding='gb18030')
for i in tqdm(range(test.shape[0])):
    test.loc[i:i,'过滤后内容分词json串'] = str(json.loads(test['过滤后内容分词json串'][i]))
test.to_csv(r'E:\Download\数据大赛\选题1\初步分词\test.csv',encoding='gb18030')
#第一部分尝试感受：目前最需要的应该是构建词表，反而不需要太过在意数据的其他属性

In [3]:
#第二部分尝试将使用进行二分类数据预处理
#1. 遍历数据，得到微博情绪的种类
sourceRoot = r'E:\Download\数据大赛\选题1\微博'
files = None
WBEmotionList = list()
for root,dirs,fs in os.walk(sourceRoot):
    files = fs
    break
for file in tqdm(files):
    fileAbsolutePath = sourceRoot+'\\'+file
    dataFile = pandas.read_csv(fileAbsolutePath,encoding='gb18030')
    WBEmotionList.extend(list(set(dataFile['微博情绪'].values.tolist())))
WBEmotionList = list(set(WBEmotionList))
saveTempFile(r'E:\Download\数据大赛\选题1\temp\WBEmotionList.json',WBEmotionList)

100%|██████████████████████████████████████████████████████████████████████████████████| 19/19 [01:38<00:00,  5.19s/it]


In [4]:
emotionsToThereEmotions = {'愤怒':'negative','恐惧':'negative','悲伤':'negative','中性':'neutral','喜悦':'positive','惊奇':'positive'}
position = 0 #临界资源
#定义线程工作
max_Threading = 9
threadLock = threading.Lock()

def getNewFileName():
    global position
    fileName = files[position]
    if position < 19:
        position += 1
        return fileName
    else:
        return None

def workFunction(id):
    nowFileName = None
    emotion = re.compile(r'(\[.*?\])')
    tag = re.compile(r'(#.*?#)')
    while True:
        threadLock.acquire()
        nowFileName = getNewFileName()
        threadLock.release()
        print(nowFileName)
        if nowFileName is None :
            break
        words = {'negative':[],'neutral':[],'positive':[]}
        tempFile = pandas.read_csv(sourceRoot+'\\'+nowFileName,encoding='gb18030')
        total = tempFile.shape[0]
        for index,item in enumerate(tempFile.values.tolist()):
           # if (index%1000) == 0:
               # print(index/total)
            oneSentenceWords = []
            if item[0] == '转发微博 ':
                oneSentenceWords.append('转发微博 ')
                item[0].replace('转发微博 ','')
            for e in emotion.findall(item[0]):
                oneSentenceWords.append(e)
                item[0].replace(e,'')
            for t in tag.findall(item[0]):
                oneSentenceWords.append(t)
                item[0].replace(t,'')
            w = pseg.cut(item[0],use_paddle=True)
            for wd, fg in w:
                oneSentenceWords.append(wd)
            for j in oneSentenceWords:#将所有词记录进结构中
                words[emotionsToThereEmotions[item[16]]].append(j)
            if int((index/total)%(0.01*total)) == 0:#记录当前状态
                with open(r'E:\Download\数据大赛\选题1\temp'+'\\'+nowFileName+'.txt','a+') as f:
                    f.write(str((index/total))+'\n')
        with open(r'E:\Download\数据大赛\选题1\第二部分分词\%d.txt'%id,'a+') as f:
            for key in words:
                for wd in words[key]:
                    f.write(wd+'\t'+key+'\n')

In [ ]:
threads = []
for i in range(max_Threading):
    threads.append(threading.Thread(target=workFunction,args=[i]))
for thread in threads:
    thread.start()
for thread in threads:
    thread.join()

2020天猫双11总成交额4982亿元.csv
3人伪造老干妈印章与腾讯签合同被刑拘.csv
《乘风破浪的姐姐》成团之夜-1.csv
《乘风破浪的姐姐》成团之夜-2.csv
《网络直播营销活动行为规范》7月1日实施.csv
上海首个小区停用丰巢快递柜.csv
冒名顶替上大学拟写入刑法.csv
司机为救婴儿连闯红灯家属拒绝作证.csv
周杰伦王俊凯成为英雄联盟代言人-1.csv


Building prefix dict from the default dictionary ...
Loading model from cache C:\Users\17797\AppData\Local\Temp\jieba.cache
Loading model cost 1.838 seconds.
Prefix dict has been built successfully.
